In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df_trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [6]:
df_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [7]:
df_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, 
Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. 
One category of liquidated damages is related to chronic problems in any category of service at the same premises. 
A chronic problem is defined as more than one missed pickup for any address. 
The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided 
correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:
⦁	What other types of complaints are there?
⦁	Are there any geospatial analysis you can do?  Which visualizations can you create?
⦁	How do metro crews compare to the contractor's performance?
⦁	How much does each trash hauler owe?
⦁	What were to total missed pickup by route?

In [ ]:
BONUS!

Missed Trash Pickups
The city is considering different ways to calculate fines. Here are two methods that are being considered. 
Calculate the total fines that would be collected by each and compare that total to the amount collected using the current method.

Alternative Method 1:
For each address, if there are three or more missed pickups within a 180-day period, damages of $1500 will be charged. 
(A fine will be levied every time three unique missed pickup dates occur within a six-month period for a single address.)

Alternative Method 2:
This method also considers the six-month window like Alternative Method 1, but each date can only be used once to support a fine. 
How will this difference impact the fines levied? (Example: If Jan 1st, Mar 3rd, Apr 8th, and Aug 9th were the only four dates a 
trash pickup was missed, the original method would result in $3000 in fines [Jan, Mar, Apr, and also Mar, Apr, Aug]. 
However, this updated method would only result in $1500 because neither Mar nor Apr can be used for another fine since they were already used.)
